# Data Understanding
by Andrea

## Autoreload

Autoreload allows the notebook to dynamically load code: if we update some helper functions *outside* of the notebook, we do not need to reload the notebook.

In [1]:
%load_ext autoreload
%autoreload 2

## Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re

import os
import sys
sys.path.append('../dataset/')

In [5]:
cyclist_df = pd.read_csv(os.path.join('dataset','cyclists.csv'))
races_df = pd.read_csv(os.path.join('dataset','races.csv'))

## Preliminary exploration (just strolling around)

### Cyclists dataset

In [6]:
cyclist_df.shape

(6134, 6)

In [7]:
cyclist_df.head()

,_url,name,birth_year,weight,height,nationality
0,bruno-surra,Bruno Surra,1964.0,NaN,NaN,Italy
1,gerard-rue,Gérard Rué,1965.0,74.0,182.0,France
2,jan-maas,Jan Maas,1996.0,69.0,189.0,Netherlands
3,nathan-van-hooydonck,Nathan Van Hooydonck,1995.0,78.0,192.0,Belgium
4,jose-felix-parra,José Félix Parra,1997.0,55.0,171.0,Spain


In [8]:
# some datatypes have to be converted
cyclist_df["birth_year"] = pd.to_numeric(cyclist_df["birth_year"], errors='coerce').astype('Int64')

cyclist_df.dtypes

_url            object
name            object
birth_year       Int64
weight         float64
height         float64
nationality     object
dtype: object

Let's process the strings, before I become crazy

In [9]:
cyclist_df["name"] = cyclist_df["name"].apply(lambda nome: ' '.join(nome.split()))

In [8]:
cyclist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6134 entries, 0 to 6133
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   _url         6134 non-null   object 
 1   name         6134 non-null   object 
 2   birth_year   6121 non-null   Int64  
 3   weight       3078 non-null   float64
 4   height       3143 non-null   float64
 5   nationality  6133 non-null   object 
dtypes: Int64(1), float64(2), object(3)
memory usage: 293.7+ KB


#### Duplicate values

The only duplicate values that it makes sense to check are the cyclists' names and identifiers

In [10]:
cyclist_df[cyclist_df["_url"].duplicated(keep="first")]

,_url,name,birth_year,weight,height,nationality


No duplicate URLs (i.e. identifiers). There are homonyms though, so one shoud be aware of this.

In [10]:
cyclist_df[cyclist_df["name"].duplicated(keep=False)]

,_url,name,birth_year,weight,height,nationality
347,andrea-peron-1,Andrea Peron,1971,70.0,183.0,Italy
1745,roman-kreuziger-sr,Roman Kreuziger,1965,NaN,NaN,Czech Republic
2235,alessandro-pozzi2,Alessandro Pozzi,1969,NaN,NaN,Italy
2601,roman-kreuziger,Roman Kreuziger,1986,65.0,183.0,Czech Republic
2682,andrea-peron,Andrea Peron,1988,70.0,178.0,Italy
2862,antonio-cabello-baena,Antonio Cabello,1990,67.0,179.0,Spain
2939,jesus-lopez23,Jesús López,1955,NaN,NaN,Spain
2953,alberto-fernandez-sainz,Alberto Fernández,1981,NaN,NaN,Spain
3238,antonio-cabello,Antonio Cabello,1956,NaN,NaN,Spain
4917,sergio-dominguez-rodriguez,Sergio Domínguez,1979,NaN,NaN,Spain


Upon manual checking, all these cyclists exist, therefore there are no duplicated rows

#### Missing values

In [11]:
cyclist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6134 entries, 0 to 6133
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   _url         6134 non-null   object 
 1   name         6134 non-null   object 
 2   birth_year   6121 non-null   Int64  
 3   weight       3078 non-null   float64
 4   height       3143 non-null   float64
 5   nationality  6133 non-null   object 
dtypes: Int64(1), float64(2), object(3)
memory usage: 293.7+ KB


We can see that there are missing values

In [12]:
n_rows = cyclist_df.shape[0]
for col in cyclist_df.columns:
    print(f"There are {n_rows - cyclist_df[col].count()} null values in the {col} column, i.e. {100*(n_rows - cyclist_df[col].count())/n_rows:.2f}% are missing")

There are 0 null values in the _url column, i.e. 0.00% are missing
There are 0 null values in the name column, i.e. 0.00% are missing
There are 13 null values in the birth_year column, i.e. 0.21% are missing
There are 3056 null values in the weight column, i.e. 49.82% are missing
There are 2991 null values in the height column, i.e. 48.76% are missing
There are 1 null values in the nationality column, i.e. 0.02% are missing


The best thing to do would be to integrate this missing data, if possible. We'll see

### Races dataset

In [13]:
races_df.shape

(589865, 18)

In [14]:
races_df.head()

,_url,name,points,uci_points,length,climb_total,profile,startlist_quality,average_temperature,date,position,cyclist,cyclist_age,is_tarmac,is_cobbled,is_gravel,cyclist_team,delta
0,tour-de-france/1978/stage-6,Tour de France,100.0,NaN,162000.0,1101.0,1.0,1241,NaN,1978-07-05 04:02:24,0,sean-kelly,22.0,True,False,False,vini-ricordi-pinarello-sidermec-1986,0.0
1,tour-de-france/1978/stage-6,Tour de France,100.0,NaN,162000.0,1101.0,1.0,1241,NaN,1978-07-05 04:02:24,1,gerrie-knetemann,27.0,True,False,False,norway-1987,0.0
2,tour-de-france/1978/stage-6,Tour de France,100.0,NaN,162000.0,1101.0,1.0,1241,NaN,1978-07-05 04:02:24,2,rene-bittinger,24.0,True,False,False,NaN,0.0
3,tour-de-france/1978/stage-6,Tour de France,100.0,NaN,162000.0,1101.0,1.0,1241,NaN,1978-07-05 04:02:24,3,joseph-bruyere,30.0,True,False,False,navigare-blue-storm-1993,0.0
4,tour-de-france/1978/stage-6,Tour de France,100.0,NaN,162000.0,1101.0,1.0,1241,NaN,1978-07-05 04:02:24,4,sven-ake-nilsson,27.0,True,False,False,spain-1991,0.0


In [15]:
races_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589865 entries, 0 to 589864
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   _url                 589865 non-null  object 
 1   name                 589865 non-null  object 
 2   points               589388 non-null  float64
 3   uci_points           251086 non-null  float64
 4   length               589865 non-null  float64
 5   climb_total          442820 non-null  float64
 6   profile              441671 non-null  float64
 7   startlist_quality    589865 non-null  int64  
 8   average_temperature  29933 non-null   float64
 9   date                 589865 non-null  object 
 10  position             589865 non-null  int64  
 11  cyclist              589865 non-null  object 
 12  cyclist_age          589752 non-null  float64
 13  is_tarmac            589865 non-null  bool   
 14  is_cobbled           589865 non-null  bool   
 15  is_gravel        

In [16]:
print(f"There are {len(races_df.select_dtypes(include="number").columns)} columns with numeric attributes.")
print(f"\t They are: {[col for col in races_df.select_dtypes(include="number").columns]}\n")
print(f"There are {len(races_df.select_dtypes(exclude="number").columns)} non-numeric attributes.")
print(f"\t They are: {[col for col in races_df.select_dtypes(exclude="number")]}")

There are 10 columns with numeric attributes.
	 They are: ['points', 'uci_points', 'length', 'climb_total', 'profile', 'startlist_quality', 'average_temperature', 'position', 'cyclist_age', 'delta']

There are 8 non-numeric attributes.
	 They are: ['_url', 'name', 'date', 'cyclist', 'is_tarmac', 'is_cobbled', 'is_gravel', 'cyclist_team']


Unlike the Cyclists database, it looks like we don't have to fix the strings here.

#### A little sanity check

In [17]:
# I've pretty much nailed the final part. 
# The [a-z0-9-]+ is a bit rough, but does the job
pattern = r"([a-z0-9-]+)/\d{4}/(prologue|result|stage-\d)"
all(races_df['_url'].apply(lambda url: bool(re.match(pattern,url))))

True

So all URLs can be described with this pattern

#### Duplicate values

The first thing that comes into my mind is to check if the races are all distinct.

In [18]:
race_names = np.sort(races_df['name'].unique())
for race in race_names:
    print(race)

Amstel Gold Race
Clasica Ciclista San Sebastian
Clásica Ciclista San Sebastian
Clásica Ciclista San Sebastián
Clásica San Sebastián
Criterium du Dauphiné
Criterium du Dauphiné Libére
Critérium du Dauphiné
Critérium du Dauphiné Libéré
Donostia San Sebastian Klasikoa
Dwars door België / À travers la Belgique
Dwars door Vlaanderen
Dwars door Vlaanderen - A travers la Flandre ME
Dwars door Vlaanderen / A travers la Flandre
Dwars door Vlaanderen / A travers la Flandre ME
E3 BinckBank Classic
E3 Harelbeke
E3 Prijs Vlaanderen
E3 Prijs Vlaanderen - Harelbeke
E3 Saxo Bank Classic
E3 Saxo Classic
E3-Prijs Harelbeke
Giro d'Italia
Giro di Lombardia
Gran Camiño
Grand Prix Cycliste de Montréal
Grand Prix Cycliste de Quebec
Grand Prix Cycliste de Québec
Il Lombardia
Itzulia Basque Country
La Flèche Wallonne
La Vuelta ciclista a España
Liège - Bastogne - Liège
Liège-Bastogne-Liège
Milano-Sanremo
Monte Paschi Eroica
Montepaschi Strade Bianche - Eroica Toscana
O Gran Camiño
Omloop Het Nieuwsblad ME
Omlo

One can already see that there are some suspects here...

In [19]:
def check_if_same(race1:str, 
                  race2:str, 
                  pattern:str=r"([a-z0-9-]+)/\d{4}/(prologue|result|stage-\d)") -> tuple:
    """Checks if two names refer to the same race, comparing the name that appears in the 
    `_url` colon. It uses the regular expression passed as `pattern` to extract the race ID

    Args:
        race1 (str): name of the first race to compare
        race2 (str): name of the second race to compare
        pattern (str, optional): The pattern against which to check the URLs. Defaults to r"([a-z0-9-]+)/\\d{4}/(prologue|result|stage-\\d)".

    Returns:
        tuple: equality (True|False), race1's ID(s), race2's ID(s)
    """
    race1_urls = races_df[races_df['name'] == race1]['_url'].unique()
    race2_urls = races_df[races_df['name'] == race2]['_url'].unique()
    # Just checks
    if len(race1_urls) == 0:
        print(f"The race name {race1} has no corresponding URLs.\n Are you sure you didn't misspell?")
        return
    if len(race2_urls) == 0:
        print(f"The race name {race2} has no corresponding URLs.\n Are you sure you didn't misspell?")
        return
    
    # This pattern matches all the races' URLs
    #pattern = r"([a-z0-9-]+)/\d{4}/(prologue|result|stage-\d)"

    def extract_race_ID(race_url:str) -> str|None:
        match = re.match(pattern,race_url)
        if match:
            # We target the name of the race
            return match.group(1)
        return None

    race_ID_1 = np.unique(np.array([extract_race_ID(url) for url in race1_urls]))
    race_ID_2 = np.unique(np.array([extract_race_ID(url) for url in race2_urls]))

    return np.array_equal(race_ID_1,race_ID_2), race_ID_1, race_ID_2


With the `check_if_same` function we can check if two slightly different names actually correspond to the same race, by comparing the first part of the associated `_url` 

In [192]:
# Initialize a list to store pairs of races that are actually the same
same_races = []

# Iterate through all pairs of race names
for i in range(len(race_names)):
    for j in range(i + 1, len(race_names)):
        race1 = race_names[i]
        race2 = race_names[j]
        # Use the check_if_same function to compare the races
        try:
            same = check_if_same(race1, race2)[0]
            if same:
                same_races.append((race1, race2))
        except TypeError:
            print(f"Caught error at races {race_names[i]} and {race_names[j]}")
        

# Print the pairs of races that are actually the same
for race1, race2 in same_races:
    print(f"The races '{race1}' and '{race2}' are actually the same.")

The races 'Clasica Ciclista San Sebastian' and 'Clásica Ciclista San Sebastian' are actually the same.
The races 'Clasica Ciclista San Sebastian' and 'Clásica Ciclista San Sebastián' are actually the same.
The races 'Clasica Ciclista San Sebastian' and 'Clásica San Sebastián' are actually the same.
The races 'Clasica Ciclista San Sebastian' and 'Donostia San Sebastian Klasikoa' are actually the same.
The races 'Clásica Ciclista San Sebastian' and 'Clásica Ciclista San Sebastián' are actually the same.
The races 'Clásica Ciclista San Sebastian' and 'Clásica San Sebastián' are actually the same.
The races 'Clásica Ciclista San Sebastian' and 'Donostia San Sebastian Klasikoa' are actually the same.
The races 'Clásica Ciclista San Sebastián' and 'Clásica San Sebastián' are actually the same.
The races 'Clásica Ciclista San Sebastián' and 'Donostia San Sebastian Klasikoa' are actually the same.
The races 'Clásica San Sebastián' and 'Donostia San Sebastian Klasikoa' are actually the same.
Th

In [204]:
print("Don't trust the following sentence!")
print(f"Allegedly, there are {len(same_races)/2} races that are the same, out of {len(race_names)} possible")

Don't trust the following sentence!
Allegedly, there are 37.0 races that are the same, out of 61 possible


It looks like many races are the same, but they changed name between years. To have the confirmation, one should check the data more carefully considering multiple sources, of course.

---

In [121]:
races_df.groupby("name")['_url'].unique()#.iloc[1]

name
Amstel Gold Race                      [amstel-gold-race/2018/result, amstel-gold-rac...
Clasica Ciclista San Sebastian        [san-sebastian/2016/result, san-sebastian/2006...
Clásica Ciclista San Sebastian                              [san-sebastian/2017/result]
Clásica Ciclista San Sebastián        [san-sebastian/2019/result, san-sebastian/1990...
Clásica San Sebastián                 [san-sebastian/1981/result, san-sebastian/1982...
                                                            ...                        
Vuelta Ciclista al País Vasco         [itzulia-basque-country/2012/stage-1, itzulia-...
Vuelta a España                       [vuelta-a-espana/2016/stage-14, vuelta-a-espan...
Vuelta al País Vasco                  [itzulia-basque-country/2007/stage-3, itzulia-...
World Championships - Road Race       [world-championship/1996/result, world-champio...
World Championships ME - Road Race    [world-championship/2002/result, world-champio...
Name: _url, Length: 61, dty

#### Missing values

In [20]:
races_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589865 entries, 0 to 589864
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   _url                 589865 non-null  object 
 1   name                 589865 non-null  object 
 2   points               589388 non-null  float64
 3   uci_points           251086 non-null  float64
 4   length               589865 non-null  float64
 5   climb_total          442820 non-null  float64
 6   profile              441671 non-null  float64
 7   startlist_quality    589865 non-null  int64  
 8   average_temperature  29933 non-null   float64
 9   date                 589865 non-null  object 
 10  position             589865 non-null  int64  
 11  cyclist              589865 non-null  object 
 12  cyclist_age          589752 non-null  float64
 13  is_tarmac            589865 non-null  bool   
 14  is_cobbled           589865 non-null  bool   
 15  is_gravel        

In [21]:
n_rows = races_df.shape[0]
for col in races_df.columns:
    print(f"There are {n_rows - races_df[col].count()} null values in the {col} column, i.e. {100*(n_rows - races_df[col].count())/n_rows:.2f}% are missing")

There are 0 null values in the _url column, i.e. 0.00% are missing
There are 0 null values in the name column, i.e. 0.00% are missing
There are 477 null values in the points column, i.e. 0.08% are missing
There are 338779 null values in the uci_points column, i.e. 57.43% are missing
There are 0 null values in the length column, i.e. 0.00% are missing
There are 147045 null values in the climb_total column, i.e. 24.93% are missing
There are 148194 null values in the profile column, i.e. 25.12% are missing
There are 0 null values in the startlist_quality column, i.e. 0.00% are missing
There are 559932 null values in the average_temperature column, i.e. 94.93% are missing
There are 0 null values in the date column, i.e. 0.00% are missing
There are 0 null values in the position column, i.e. 0.00% are missing
There are 0 null values in the cyclist column, i.e. 0.00% are missing
There are 113 null values in the cyclist_age column, i.e. 0.02% are missing
There are 0 null values in the is_tarma

Maybe some missing values can be integrated from other sources. Who knows

#### Consistency checks

I'm curious to check if some stages/tracks have multiple terrain

I create a new feature with just the race name and its stage. This is to "isolate" the year

In [22]:
def extract_name_stage(url:str,
              pattern:str=r"([a-z0-9-]+)/\d{4}/(prologue|result|stage-\d)") -> str:
    match = re.match(pattern,url)
    if match:
        return f"{match.group(1)}_{match.group(2)}"
    else:
        return None


races_df_copy = pd.DataFrame.copy(races_df)

races_df_copy["name_stage"] = races_df_copy["_url"].apply(extract_name_stage)


Then I group the dataframe based on this new column, and get the unique values for `is_X`, for each group.

In [39]:
terrain_grouped_data = races_df_copy.groupby('name_stage').agg({'is_tarmac': 'unique',
                                                                'is_cobbled': 'unique',
                                                                'is_gravel': 'unique'})
terrain_grouped_data

,is_tarmac,is_cobbled,is_gravel
name_stage,,,
amstel-gold-race_result,[False],[False],[False]
dauphine_prologue,[True],[False],[False]
dauphine_stage-1,[True],[False],[False]
dauphine_stage-2,[True],[False],[False]
dauphine_stage-3,[True],[False],[False]
...,...,...,...
vuelta-a-espana_stage-6,[True],[False],[False]
vuelta-a-espana_stage-7,[True],[False],[False]
vuelta-a-espana_stage-8,[True],[False],[False]


In [76]:
def count_n_terrains(terrain:str) -> int:
    try:
        n = terrain_grouped_data[f'is_{terrain}'].apply(lambda lista: lista[0]).sum()
        return n
    except KeyError:
        print(f"terrain value {terrain} is invalid.")
        print("Only options are 'tarmac', 'cobbled', 'gravel'")

for terrain in ['tarmac', 'cobbled','gravel']:
    print(f'Out of {terrain_grouped_data.shape[0]} (unique) "atomic competitions", {count_n_terrains(terrain)} have {terrain}')

Out of 118 (unique) "atomic competitions", 103 have tarmac
Out of 118 (unique) "atomic competitions", 0 have cobbled
Out of 118 (unique) "atomic competitions", 0 have gravel


This is highly suspicious.

In [80]:
for terrain in ['tarmac', 'cobbled','gravel']:
    n = races_df[f'is_{terrain}'].sum()
    print(f"In the big dataframe, out of {races_df.shape[0]}, {n} have {terrain}")

In the big dataframe, out of 589865, 536042 have tarmac
In the big dataframe, out of 589865, 0 have cobbled
In the big dataframe, out of 589865, 0 have gravel


Let's check if the same race has been given different values for the presence of some terrain (maybe because of errors).

In [56]:
def check_consistent_terrain(terrain:str) -> bool:
    try:
        # We check if there is only one value (either True or False)
        biglist = terrain_grouped_data[f'is_{terrain}'].apply(lambda lista: bool(len(lista)==1))
        return all(biglist)
    except KeyError:
        print(f"terrain value {terrain} is invalid.")
        print("Only options are 'tarmac', 'cobbled', 'gravel'")

for terrain in ['tarmac', 'cobbled','gravel']:
    print(check_consistent_terrain(terrain))


True
True
True


It looks like it's not the case. This means that the same "atomic competitions" (i.e. stages, or subdivisions in general) of the same race have been given the same terrain over the year.

Let's try to find the races that have more than one terrain (if there are any)

In [67]:
no_terrain_races, multiple_terrain_races = [], []

for group_key, group_df in races_df_copy.groupby('name_stage'):
    terrains = group_df[['is_tarmac','is_cobbled','is_gravel']].agg({'is_tarmac': 'unique',
                                                                'is_cobbled': 'unique',
                                                                'is_gravel': 'unique'})
    if terrains.sum() == 0:
        print(f"Group {group_key} has no terrains")
        print(terrains)
        print()
        no_terrain_races.append(group_key)
    elif terrains.sum() in [2,3]:
        print(f"Group {group_key} has multiple terrains")
        print(terrains)
        print()
        multiple_terrain_races.append(group_key)
    

Group amstel-gold-race_result has no terrains
is_tarmac     [False]
is_cobbled    [False]
is_gravel     [False]
dtype: object

Group dwars-door-vlaanderen_result has no terrains
is_tarmac     [False]
is_cobbled    [False]
is_gravel     [False]
dtype: object

Group e3-harelbeke_result has no terrains
is_tarmac     [False]
is_cobbled    [False]
is_gravel     [False]
dtype: object

Group gp-montreal_result has no terrains
is_tarmac     [False]
is_cobbled    [False]
is_gravel     [False]
dtype: object

Group gp-quebec_result has no terrains
is_tarmac     [False]
is_cobbled    [False]
is_gravel     [False]
dtype: object

Group il-lombardia_result has no terrains
is_tarmac     [False]
is_cobbled    [False]
is_gravel     [False]
dtype: object

Group la-fleche-wallone_result has no terrains
is_tarmac     [False]
is_cobbled    [False]
is_gravel     [False]
dtype: object

Group liege-bastogne-liege_result has no terrains
is_tarmac     [False]
is_cobbled    [False]
is_gravel     [False]
dtype: ob

In [68]:
print(f"Races with no terrain: {no_terrain_races}")
print(f"Races with multiple terrains: {multiple_terrain_races}")

Races with no terrain: ['amstel-gold-race_result', 'dwars-door-vlaanderen_result', 'e3-harelbeke_result', 'gp-montreal_result', 'gp-quebec_result', 'il-lombardia_result', 'la-fleche-wallone_result', 'liege-bastogne-liege_result', 'milano-sanremo_result', 'omloop-het-nieuwsblad_result', 'paris-roubaix_result', 'ronde-van-vlaanderen_result', 'san-sebastian_result', 'strade-bianche_result', 'world-championship_result']
Races with multiple terrains: []
